# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ing. en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**: Real-time log analyzer

**Fecha**: 3 de abril del 2025

**Nombre del Estudiante**: Marco Albanese, Vicente Siloe

**Profesor**: Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Real-time log analyzer") \
    .master("spark://f8e470126b03:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [ ]:
from equipo_mcqueen.traffic_query_listener import TrafficListener

spark.streams.addListener(TrafficListener())

### Description

- In teams, build a streaming pipeline that monitors a directory for a **simulated** server log files, analyzes error patterns in real time, and triggers alerts for critical issues (eg., repeated 500 errors).
- The notebook with your solution should be named with the following pattern: **spark_cluster/notebooks/labs/lab07/lab07_<team_name>.ipynb**
- You need to create one script that generates random log entries (using bash or python). Add this script in the lib folder of your team.
- Submit to Canvas a PR link including the script you created to generate random log entries and the Jupyter Notebook with your solution. In the Notebook should be visible at least **three micro batches of the streaming**.


In [47]:
log_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/log_data/input/")

In [48]:
from pyspark.sql.functions import split

logs_df = log_lines.select(split(log_lines.value, " ").alias("log_array"))
logs_df = logs_df.withColumn("date", logs_df["log_array"].getItem(0).cast("date"))
logs_df = logs_df.withColumn("time", logs_df["log_array"].getItem(1).cast("string"))
logs_df = logs_df.withColumn("level", logs_df["log_array"].getItem(2).cast("string"))
logs_df = logs_df.withColumn("message", logs_df["log_array"].getItem(3).cast("string"))
logs_df = logs_df.withColumn("server_node", logs_df["log_array"].getItem(4).cast("string"))

logs_df.printSchema()

root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- level: string (nullable = true)
 |-- message: string (nullable = true)
 |-- server_node: string (nullable = true)



Filter by ERROR

In [49]:
logs_df = logs_df.filter(logs_df.level == "ERROR")

In [50]:
query = logs_df.writeStream \
    .outputMode("append") \
    .trigger(processingTime="2 seconds") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

Query started: a4a3e10a-3801-4fd6-8a27-4e8ba9b74b14
Query started: a4a3e10a-3801-4fd6-8a27-4e8ba9b74b14
Query started: a4a3e10a-3801-4fd6-8a27-4e8ba9b74b14
-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------------------------------------+----------+--------+-----+-------------------------+-------------+
|log_array                                                               |date      |time    |level|message                  |server_node  |
+------------------------------------------------------------------------+----------+--------+-----+-------------------------+-------------+
|[2025-04-02, 20:07:32, ERROR, 500-Internal-Server-Error, server-node-2] |2025-04-02|20:07:32|ERROR|500-Internal-Server-Error|server-node-2|
|[2025-04-02, 20:07:32, ERROR, 500-Internal-Server-Error, server-node-2] |2025-04-02|20:07:32|ERROR|500-Internal-Server-Error|server-node-2|
|[2025-04-02, 20:07:32, ERROR, 500-Interna

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

Query made progress: {
  "id" : "a4a3e10a-3801-4fd6-8a27-4e8ba9b74b14",
  "runId" : "ed7ccddf-3040-42c7-a995-b8dc46bdcb05",
  "name" : null,
  "timestamp" : "2025-04-03T02:08:16.000Z",
  "batchId" : 19,
  "numInputRows" : 49,
  "inputRowsPerSecond" : 24.5,
  "processedRowsPerSecond" : 357.6642335766423,
  "durationMs" : {
    "addBatch" : 50,
    "commitOffsets" : 20,
    "getBatch" : 22,
    "latestOffset" : 22,
    "queryPlanning" : 4,
    "triggerExecution" : 137,
    "walCommit" : 19
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/home/jovyan/notebooks/data/log_data/input]",
    "startOffset" : {
      "logOffset" : 18
    },
    "endOffset" : {
      "logOffset" : 19
    },
    "latestOffset" : null,
    "numInputRows" : 49,
    "inputRowsPerSecond" : 24.5,
    "processedRowsPerSecond" : 357.6642335766423
  } ],
  "sink" : {
    "description" : "org.apache.spark.sql.execution.streaming.ConsoleTable$@234cb5f0",
    "numOutputRows" : 6
  

In [ ]:
query.stop()

Query terminated: c371751e-7a00-46fd-a8d9-6e7a6d620375
Query terminated: c371751e-7a00-46fd-a8d9-6e7a6d620375
Query terminated: c371751e-7a00-46fd-a8d9-6e7a6d620375


In [ ]:
sc.stop()